## Analyze STDP update dynamics vs neuron firing rates
* It's important that the polarities of change are consistent, not so much magnitude

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from spikey.synapse import *
from spikey.neuron import *


def onehot(value, buckets):
    output = np.zeros(buckets)
    output[value] = 1
    return output

SYNAPSE = RLSTDPET
NEURON = Neuron

In [ ]:
## vs Presynaptic rate
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]  # hz, number of fires / interval -- evenly spaced

for rate2 in RATES:
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()

        a = np.zeros(LENGTH)
        a[::LENGTH // rate1] = 1.

        b = np.zeros(LENGTH)
        b[::LENGTH // rate2] = 1.

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"pre={rate2}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs 2 Presynaptic rates
config = {
    'n_neurons': 3,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1, 1])

synapses.reset()

rate1 = 75
a = np.zeros(LENGTH)
a[::LENGTH // rate1] = 1.

rate2 = 50
rate3 = 100

b = np.zeros(LENGTH)
b[::LENGTH // rate2] = 1.

c = np.zeros(LENGTH)
c[::LENGTH // rate3] = 1.

full_spike_log = np.array([a, b, c]).T

for i in range(1, LENGTH):
    synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

print(synapses.trace)

In [ ]:
## vs Presynaptic rate w/ offset
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]  # hz, number of fires / interval -- evenly spaced

for rate2 in RATES:
    X, Y = [], []

    for rate1 in np.arange(5, 150, 5):
        synapses.reset()

        a = np.zeros(LENGTH)
        a[3::LENGTH // rate1] = 1.

        b = np.zeros(LENGTH)
        b[::LENGTH // rate2] = 1.

        full_spike_log = np.array([a, b]).T
        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"pre={rate2}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs Presynaptic rate - Uniform Rates
# ideally uniform will be a flat line, any randomness should create all possible coincidences
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]  # hz, number of fires / interval -- evenly spaced

for rate2 in RATES:
    X, Y = [], []

    for rate1 in np.arange(5, 150, 5):
        synapses.reset()

        a = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate1 / LENGTH)
        b = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate2 / LENGTH)

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"pre={rate2}")
    # plt.plot(np.unique(X), np.poly1d(np.polyfit(X, Y, 1))(np.unique(X)), label=f"pre={rate2}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs Presynaptic rate - Uniform Rates not in hz
# ideally uniform will be a flat line, any randomness should create all possible coincidences
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = np.arange(0, 1.1, .1)
RATES = np.round(RATES, decimals=4)  # Fix numerical erros

for rate2 in RATES:
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()

        a = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate1)
        b = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate2)

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"pre={rate2}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs Presynaptic rate - Poisson Rates not in hz
# ideally uniform will be a flat line, any randomness should create all possible coincidences
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 150,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': None,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = np.arange(0, .1, .01)
RATES = np.round(RATES, decimals=4)  # Fix numerical erros

for rate2 in RATES:
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()
        
        LAMBDA = .5
        a = np.int_(np.random.poisson(LAMBDA, size=LENGTH) <= rate1)
        b = np.int_(np.random.uniform(LAMBDA, size=LENGTH) <= rate2)

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"pre={rate2}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs STDP Window
# ideally uniform will be a flat line, any randomness should create all possible coincidences
LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]

for window in [2, 10, 50, 100, 150, 200]:
    config = {
        'n_neurons': 2,
        'n_inputs': 0,
        'stdp_window': window,
        'learning_rate': 1,
        'max_weight': 1,
        'trace_decay': None,
    }
    synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

    X, Y = [], []

    for rate in RATES:
        synapses.reset()

        a = np.zeros(LENGTH)
        a[::LENGTH // rate] = 1.

        b = np.zeros(LENGTH)
        b[::LENGTH // 100] = 1.

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"{window}")

plt.title(f"rate vs window dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs STDP Window w/ very large sizes
# -- seemingly, the stdp_window has to be larger than the processing time to stop growing.
LENGTH = 10000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]

for window in [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]:
    config = {
        'n_neurons': 2,
        'n_inputs': 0,
        'stdp_window': window,
        'learning_rate': 1,
        'max_weight': 1,
        'trace_decay': None,
    }
    synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

    X, Y = [], []

    for rate in RATES:
        synapses.reset()

        a = np.zeros(LENGTH)
        a[::LENGTH // rate] = 1.

        b = np.zeros(LENGTH)
        b[::LENGTH // 100] = 1.

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"{window}")

plt.title(f"rate vs window dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
## vs STDP Window - Uniform Rates
# ideally uniform will be a flat line, any randomness should create all possible coincidences
LENGTH = 1000
INHIBITORIES = np.array([1, 1])

RATES = [1, 25, 50, 75, 100, 125, 150]

for window in [2, 10, 50, 100, 150, 200]:
    config = {
        'n_neurons': 2,
        'n_inputs': 0,
        'stdp_window': window,
        'learning_rate': 1,
        'max_weight': 1,
        'trace_decay': None,
    }
    synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

    X, Y = [], []

    for rate in RATES:
        synapses.reset()

        a = np.int_(np.random.uniform(0, 1, size=LENGTH) <= rate / LENGTH)
        b = np.int_(np.random.uniform(0, 1, size=LENGTH) <= 100 / LENGTH)

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"{window}")

plt.title(f"rate vs window dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()

In [ ]:
##  input mv vs w vs rate
# NOTE results may be different with spiking of neuron
LENGTH = 1000

config = {
    'magnitude': 1,
    'n_neurons': 1,
    'neuron_pct_inhibitory': 0,
    'potential_decay': .2,
    'prob_rand_fire': 0,
    'prob_rand_fire_setter': None,
    'refractory_period': 2,
    'resting_mv': 0,
    'spike_delay': 0,
    'firing_threshold': 1,
}
for mv in np.arange(0, .5, .05):
    neurons = NEURON(**config)

    X, Y = [], []

    for w in np.arange(0, 1.1, .1):
        neurons.reset()

        spikes = 0
        for i in range(LENGTH):
            neurons.update()

            neurons += mv * w

            spikes += neurons >= config['firing_threshold']

        X.append(w)
        Y.append(spikes / LENGTH)

    plt.plot(X, Y, label=f"mv={mv:.2f}")

plt.title(f"rate vs window dW")
plt.xlabel('w')
plt.ylabel('rate')
plt.legend()

In [ ]:
## rate vs potential decays!
LENGTH = 1000

config = {
    'magnitude': 1,
    'n_neurons': 1,
    'neuron_pct_inhibitory': 0,
    'potential_decay': .2,
    'prob_rand_fire': 0,
    'prob_rand_fire_setter': None,
    'refractory_period': 2,
    'resting_mv': 0,
    'spike_delay': 0,
    'firing_threshold': 1,
}

for decay in np.arange(0, 1.1, .1):
    config.update({
        'potential_decay': decay
    })
    neurons = NEURON(**config)

    X, Y = [], []
    for mv in np.arange(0, 1, .1):
        neurons.reset()

        spikes = 0
        for i in range(LENGTH):
            neurons.update()

            neurons += mv * 1

            spikes += neurons >= config['firing_threshold']

        X.append(mv)
        Y.append(spikes / LENGTH)

    plt.plot(X, Y, label=f"decay={decay:.2f}")

plt.title(f"rate vs window dW")
plt.xlabel('mv')
plt.ylabel('rate')
plt.legend()

In [ ]:
## vs Presynaptic rate in small n_processing steps time
config = {
    'n_neurons': 2,
    'n_inputs': 0,
    'stdp_window': 20,
    'learning_rate': 1,
    'max_weight': 1,
    'trace_decay': .2,
}
synapses = SYNAPSE(np.zeros((config['n_inputs'] + config['n_neurons'], config['n_neurons'])), **config)

LENGTH = 100
INHIBITORIES = np.array([1, 1])

RATES = [0, 1, 10, 20, 30, 40, 50]  # hz, number of fires / interval -- evenly spaced

for rate2 in RATES:
    X, Y = [], []

    for rate1 in RATES:
        synapses.reset()

        a = np.zeros(LENGTH)
        if rate1:
            a[::LENGTH // rate1] = 1.

        b = np.zeros(LENGTH)
        if rate2:
            b[::LENGTH // rate2] = 1.

        full_spike_log = np.array([a, b]).T

        for i in range(1, LENGTH):
            synapses._apply_stdp(full_spike_log[:i], INHIBITORIES)

        X.append(rate1)
        Y.append(synapses.trace[1, 0])

    plt.plot(X, Y, label=f"pre={rate2}")

plt.title(f"pre - post rate dW")
plt.xlabel('rate')
plt.ylabel('trace')
plt.legend()